In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn_benchmarks.utils.plotting import make_hover_template, order_columns

In [ ]:
df = pd.read_csv("results/benchmarking/sklearn_KNeighborsClassifier_kd_tree.csv")

In [ ]:
df = df.query("function == 'predict'")

In [ ]:
df = df.fillna(value={"use_onnx_runtime": False})

In [ ]:
merged_df = df.query("use_onnx_runtime == True").merge(
    df.query("use_onnx_runtime == False"),
    on=["hyperparams_digest", "dataset_digest"],
    how="inner",
    suffixes=["", "_onnx"],
)

In [ ]:
merged_df = merged_df.dropna(axis=1)
columns_to_drop = merged_df.filter(regex="_onnx$").columns.tolist()
columns_to_drop = filter(lambda col: "mean" not in col and "stdev" not in col, columns_to_drop)
merged_df.drop(
    columns_to_drop,
    axis=1,
    inplace=True,
)

In [ ]:
merged_df["speedup"] = merged_df["mean"] / merged_df["mean_onnx"]

In [ ]:
merged_df["stdev_speedup"] = merged_df["speedup"] * (
    np.sqrt(
        (merged_df["stdev"] / merged_df["mean"]) ** 2
        + (merged_df["stdev_onnx"] / merged_df["mean_onnx"]) ** 2
    )
)

In [ ]:
x = [i + 1 for i in range(len(merged_df))]

In [ ]:
y = merged_df["speedup"]

# `scikit-learn` vs. `ONNX Runtime (Microsoft)` prediction benchmarks

In [ ]:
fig = go.Figure(
    [
        go.Bar(
            x=x,
            y=y,
            hovertemplate=make_hover_template(merged_df),
            customdata=merged_df[order_columns(merged_df)].values,
        )
    ]
)
fig.show()